In [3]:
#task 1 and 2
#exploring dataset provided and converting it to csv file
#loading pandas library
#reading csv file
import numpy as np
import pandas as pd
transfusion=pd.read_csv('transfusion.csv')

In [4]:
#printing first 5 lines from dataset

transfusion.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [5]:
#task 3
#inspecting dataset structure

transfusion.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Recency (months)                            748 non-null    int64
 1   Frequency (times)                           748 non-null    int64
 2   Monetary (c.c. blood)                       748 non-null    int64
 3   Time (months)                               748 non-null    int64
 4   whether he/she donated blood in March 2007  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB


In [6]:
#task 4 
#renaming a column and viewing first 2 rows

transfusion.rename(columns={"whether he/she donated blood in March 2007":"target"},inplace=True)

transfusion.head(2)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),target
0,2,50,12500,98,1
1,0,13,3250,28,1


In [7]:
#task 5
#print target incidence and unique values
#rounded to 3 decimal values

transfusion['target'].value_counts(normalize=True).round(3)

0    0.762
1    0.238
Name: target, dtype: float64

In [8]:
#task 6 
#spliting data into train and test sets using sklearn
#printing first 2rows of x_train set

from sklearn.model_selection import train_test_split
X=transfusion.iloc[:,:-1]
y=transfusion.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)
X_train.head(2)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
529,2,6,1500,22
271,16,7,1750,28


In [9]:
#task 7
#importing necessary libraries

from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

# Create an instance of TPOTClassifier with light configuration
tpot = TPOTClassifier( 
    generations=5, 
    population_size=20, 
    random_state=42, 
    scoring='roc_auc',  # Optimize for ROC AUC score
    config_dict='TPOT light'  # Use TPOT's light configuration with fast models and preprocessors
)

# Fit the model to the training data
tpot.fit(X_train, y_train)

# Predict using the fitted model
y_pred = tpot.predict(X_test)

# Calculate and print the AUC score, rounded to 4 decimal places
tpot_auc_score = roc_auc_score(y_test, y_pred)
print(f"tpot_auc_score: {tpot_auc_score:.4f}")

# Print the pipeline steps
print("\nBest pipeline steps:")
for idx, step in enumerate(tpot.fitted_pipeline_.steps):
    print(f"Step {idx}: {step}")

C:\Users\Ravi Sharma\AppData\Local\Programs\Python\Python310\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


tpot_auc_score: 0.5424

Best pipeline steps:
Step 0: ('zerocount', ZeroCount())
Step 1: ('stackingestimator', StackingEstimator(estimator=GaussianNB()))
Step 2: ('logisticregression', LogisticRegression(C=0.01, random_state=42))


In [10]:
#task 8
#Print X_train's variance

X_train_variance = X_train.var().round(3)

# Print the column-wise variance rounded to 3 decimal places
print(X_train_variance)

Recency (months)              67.473
Frequency (times)             32.624
Monetary (c.c. blood)    2039028.756
Time (months)                599.952
dtype: float64


In [11]:
#task 9

# Copy X_train and X_test into new DataFrames
X_train_normed = X_train.copy()
X_test_normed = X_test.copy()

# Calculate variance for each column in X_train
variances = X_train.var()

# Find the column with the highest variance
col_to_normalize = variances.idxmax()

## Normalize the column with the highest variance 
for df in [X_train_normed, X_test_normed]:
    df[col_to_normalize] = np.log(df[col_to_normalize])

# Drop the column with the highest variance
X_train_normed = X_train_normed.drop(columns=[col_to_normalize])
X_test_normed = X_test_normed.drop(columns=[col_to_normalize])

# Print the variance of X_train_normed (rounded to 3 decimal places)
print(X_train_normed.var().round(3))

Recency (months)      67.473
Frequency (times)     32.624
Time (months)        599.952
dtype: float64


In [12]:
#task 10

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

logreg=LogisticRegression(random_state=42)

#train the logistic regression model
logreg.fit(X_train,y_train)

#predict using trained model
y_pred=logreg.predict(X_test)

#calculate and print AUC score
score=roc_auc_score(y_test,y_pred)
print(f"logreg_auc_score: {score:}")

logreg_auc_score: 0.5423976608187135


In [13]:
#task 11

from operator import itemgetter
from sklearn.metrics import roc_auc_score

models = [
    ("TPOT Model", tpot_auc_score),
    ("Logistic Regression", score)
]

# Sort the list of (model_name, model_score) pairs based on AUC score from highest to lowest
sorted_models = sorted(models, key=itemgetter(1), reverse=True)

# Print sorted models and their AUC scores
for model_name, model_score in sorted_models:
    print(f"{model_name}: AUC Score = {model_score:}")

TPOT Model: AUC Score = 0.5423976608187135
Logistic Regression: AUC Score = 0.5423976608187135
